In [1]:
library(grid)
library(rpart)
library(rpart.plot)
library(partykit)
library(lattice)
library(ggplot2)
library(caret)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
set.seed(1)

In [3]:
jRoadType <- c("都市間高速", "都市高速", "有料道路", "国道", "県道", "主要地方道", "一般道1、一般道2、一般道3", "その他")
jHighSpeeds <- c("都市間高速", "都市高速", "有料道路")

In [4]:
printf <- function(...) cat(sprintf(...))

In [5]:
# valid columns when high speed
invalids <- c('Time', 'Longitude', 'Latitude', 'Brake', 'Accel', 'RoadType', 'flag', 'TimeToCollision', "RoadFactor")
isValidColumn <- function(x) ! x %in% invalids

In [6]:
CV <- function(dfx, expr, verbose=FALSE) {        
    folds <- createFolds(dfx$flag, k=5)
    
    count <- 1
    errs <- c()
    reds <- c()
    blues <- c()
    for (ids in folds) {
        train <- dfx[-ids, ]
        test <- dfx[ids, ]
        # fit <- rpart(expr, data=train, method="class")
        fit <- rpart(expr, data=train, method="class", cp=0.022)
        p <- predict(fit, newdata=test)
        predictedFlags <- colnames(p)[max.col(p, ties.method = "first")]
        
        if (verbose) {
            printf("Fold%d\n", count)
            result <- correctVsPredict(test, predictedFlags, verbose)
            reds <- c(reds, result[1])
            blues <- c(blues, result[2])
            count <- count + 1
            printf("\n")
        } else {
            result <- correctVsPredict(test, predictedFlags, verbose)
            reds <- c(reds, result[1])
            blues <- c(blues, result[2])
        }
        
        nerr <- sum((predictedFlags == test$flag) == FALSE)
        errs <- c(errs, nerr / nrow(test))
    }
    
    return(c(mean(errs), mean(reds), mean(blues)))
}

In [7]:
correctVsPredict <- function(test, predictedFlags, verbose=FALSE) {    
    # for All
    # nTests <- nrow(test)
    # nCorrectAll <- sum((predictedFlags == test$flag) == TRUE)
    # printf("As fo All: correct/all = %d/%d = %f\n", nCorrectAll, nTests, nCorrectAll / nTests)
    
    # for Red
    predictedRedRows <- test[predictedFlags == "Red", ]
    nCorrectReds <- sum((predictedRedRows$flag == 'Red') == TRUE)
    nPredictedReds <- nrow(predictedRedRows)
    
    # for Blue
    predictedBlueRows <- test[predictedFlags == "Blue", ]
    nCorrectBlues <- sum((predictedBlueRows$flag == 'Blue') == TRUE)
    nPredictedBlues <- nrow(predictedBlueRows)
        
    if (verbose) {
        printf("As for Red: correct/predict = %d/%d = %f\n", nCorrectReds, nPredictedReds, nCorrectReds / nPredictedReds)
        printf("As for Blue: correct/predict = %d/%d = %f\n", nCorrectBlues, nPredictedBlues, nCorrectBlues / nPredictedBlues)  
    }    
    
    c(nCorrectReds/nPredictedReds, nCorrectBlues/nPredictedBlues)
}

In [8]:
printRedRatios <- function(dfx) {
    nRed <- nrow(dfx[dfx$flag == "Red", ])
    nAll <- nrow(dfx)
    printf("Red/All = %d/%d = %f\n", nRed, nAll, nRed/nAll)
    printf("1 - Red/All = %d/%d = %f\n", nAll - nRed, nAll, 1 - nRed/nAll)
}

In [9]:
createFormula <- function(invalids, allFeatures) {
    isValidColumn <- function(x) ! x %in% invalids
    
    features <- Filter(isValidColumn, allFeatures)
    return(paste("flag ~ ", paste(features, collapse=" + ")))
}

# Predict Reds

In [10]:
df3 <- read.csv("../data/middle/sp5.csv", stringsAsFactors=FALSE)

In [11]:
df3$flag[df3$flag == "RedA"] <- "Red"
df3$flag[df3$flag == "RedB"] <- "Red"
df3$flag[df3$flag == "BlueA"] <- "Blue"
df3$flag[df3$flag == "BlueB"] <- "Blue"
df3$flag <- as.factor(df3$flag)

In [12]:
allFeatures <- c(colnames(df3))

In [13]:
roadTypes <- unique(df3$RoadType)

In [14]:
for (i in roadTypes) {
    printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
    dfx <- df3[df3$RoadType == i, ]
    expr <- createFormula(invalids, allFeatures)
    result <- CV(dfx, expr, verbose=FALSE)
    printRedRatios(dfx)
    printf("Red: Mean correct/predict = %f\n", result[2])
    printf("Blue: Mean correct/predict = %f\n", result[3])
    printf("CV value: %f", result[1])
    printf("\n\n")
}

RoadType: 7 (その他)
Red/All = 25/27 = 0.925926
1 - Red/All = 2/27 = 0.074074
Red: Mean correct/predict = 0.933333
Blue: Mean correct/predict = NaN
CV value: 0.066667

RoadType: 6 (一般道1、一般道2、一般道3)
Red/All = 90/189 = 0.476190
1 - Red/All = 99/189 = 0.523810
Red: Mean correct/predict = 0.602468
Blue: Mean correct/predict = 0.628633
CV value: 0.391465

RoadType: 4 (県道)
Red/All = 36/71 = 0.507042
1 - Red/All = 35/71 = 0.492958
Red: Mean correct/predict = 0.483175
Blue: Mean correct/predict = 0.435952
CV value: 0.533333

RoadType: 5 (主要地方道)
Red/All = 154/303 = 0.508251
1 - Red/All = 149/303 = 0.491749
Red: Mean correct/predict = 0.595961
Blue: Mean correct/predict = 0.585389
CV value: 0.409126

RoadType: 3 (国道)
Red/All = 202/324 = 0.623457
1 - Red/All = 122/324 = 0.376543
Red: Mean correct/predict = 0.682341
Blue: Mean correct/predict = 0.501794
CV value: 0.376202

RoadType: 0 (都市間高速)
Red/All = 55/81 = 0.679012
1 - Red/All = 26/81 = 0.320988
Red: Mean correct/predict = 0.695455
Blue: Mean corr

# Plot Tree

In [17]:
set.seed(1)

In [18]:
for (i in roadTypes) {
    printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
    dfx <- df3[df3$RoadType == i, ]
    expr <- createFormula(invalids, allFeatures)
    
    folds <- createFolds(dfx$flag, k=10)
    
    for (ids in folds) {
        train <- dfx[-ids, ]
        test <- dfx[ids, ]
        fit <- rpart(expr, data=train, method="class", cp=0.022)
        
        if (i != 7) {
            fname = paste("Dtree-", i, ".png", sep="")
            png(fname, height=960, width=960, res=144)
            plot(fit, uniform=TRUE, main="Classification Tree")
            text(fit, use.n=TRUE, all=TRUE, cex=.6)
            dev.off()
        }
    }
}

RoadType: 7 (その他)
RoadType: 6 (一般道1、一般道2、一般道3)
RoadType: 4 (県道)
RoadType: 5 (主要地方道)
RoadType: 3 (国道)
RoadType: 0 (都市間高速)
RoadType: 2 (有料道路)


# Variable importance

In [24]:
set.seed(1)

In [26]:
for (i in roadTypes) {
    printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
    dfx <- df3[df3$RoadType == i, ]
    expr <- createFormula(invalids, allFeatures)
    
    train <- dfx
    fit <- rpart(expr, data=train, method="class", cp=0.022)
    if (i != 7) {
        summary(fit)
    }
}

RoadType: 7 (その他)
RoadType: 6 (一般道1、一般道2、一般道3)
Call:
rpart(formula = expr, data = train, method = "class", cp = 0.022)
  n= 189 

          CP nsplit rel error    xerror       xstd
1 0.27777778      0 1.0000000 1.0000000 0.07628962
2 0.07777778      1 0.7222222 0.8777778 0.07534188
3 0.06666667      2 0.6444444 0.8666667 0.07520304
4 0.05555556      3 0.5777778 0.8333333 0.07473252
5 0.02777778      4 0.5222222 0.8111111 0.07437329
6 0.02222222      6 0.4666667 0.7777778 0.07376479
7 0.02200000      7 0.4444444 0.8222222 0.07455751

Variable importance
            Pitch      DiffAvgSpeed        DistSignal             Speed 
               19                14                12                10 
    AheadDistance     SteeringAngle      CurveAverage AccelerationSpeed 
                7                 7                 6                 4 
         MaxSpeed          Curve100          PathType   AverageVelocity 
                4                 3                 3                 2 
   

# Divide More ...

In [ ]:
df3 %>% group_by(RoadType) %>% summarize(counts=n())

# Threshold = 10, 30

In [ ]:
set.seed(1)

In [ ]:
# Not Use
dfx <- df3[df3$RoadType == i, ]
folds <- createFolds(dfx$flag, k=5)

In [ ]:
verbose = TRUE

In [ ]:
i = 6
dfx <- df3[df3$RoadType == i, ]
folds <- createFolds(dfx$flag, k=5)

In [ ]:
i = 4
dfx <- df3[df3$RoadType == i, ]
folds <- createFolds(dfx$flag, k=5)

In [ ]:
i = 5
threshold1 <- 10
threshold2 <- 30
printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
dfx <- df3[df3$RoadType == i, ]
expr <- createFormula(invalids, allFeatures)
dfx1 <- dfx %>% filter(AverageVelocity <= threshold1)
dfx2 <- dfx %>% filter(AverageVelocity > threshold1, AverageVelocity <= threshold2)
dfx3 <- dfx %>% filter(AverageVelocity > threshold2)

result1 <- CV(dfx1, expr, verbose=verbose)        
printf("AverageVelocity <= %d:\n", threshold1)
printRedRatios(dfx1)        
printf("Red: Mean correct/predict = %f\n", result1[2])
printf("Blue: Mean correct/predict = %f\n", result1[3])
printf("CV value: %f", result1[1])        
printf("\n\n")
        
result2 <- CV(dfx2, expr, verbose=verbose)
printf("%d < AverageVelocity <= %d:\n", threshold1, threshold2)
printRedRatios(dfx2)
printf("Red: Mean correct/predict = %f\n", result2[2])
printf("Blue: Mean correct/predict = %f\n", result2[3])
printf("CV value: %f", result2[1])       
printf("\n\n")

result3 <- CV(dfx3, expr, verbose=verbose)        
printf("AverageVelocity > %d:\n", threshold2)
printRedRatios(dfx3)
printf("Red: Mean correct/predict = %f\n", result2[2])
printf("Blue: Mean correct/predict = %f\n", result2[3])
printf("CV value: %f", result2[1])       
printf("\n\n")

In [ ]:
i = 3
threshold1 <- 10
threshold2 <- 30
printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
dfx <- df3[df3$RoadType == i, ]
expr <- createFormula(invalids, allFeatures)
dfx1 <- dfx %>% filter(AverageVelocity <= threshold1)
dfx2 <- dfx %>% filter(AverageVelocity > threshold1, AverageVelocity <= threshold2)
dfx3 <- dfx %>% filter(AverageVelocity > threshold2)

result1 <- CV(dfx1, expr, verbose=verbose)        
printf("AverageVelocity <= %d:\n", threshold1)
printRedRatios(dfx1)        
printf("Red: Mean correct/predict = %f\n", result1[2])
printf("Blue: Mean correct/predict = %f\n", result1[3])
printf("CV value: %f", result1[1])        
printf("\n\n")
        
result2 <- CV(dfx2, expr, verbose=verbose)
printf("%d < AverageVelocity <= %d:\n", threshold1, threshold2)
printRedRatios(dfx2)
printf("Red: Mean correct/predict = %f\n", result2[2])
printf("Blue: Mean correct/predict = %f\n", result2[3])
printf("CV value: %f", result2[1])       
printf("\n\n")

result3 <- CV(dfx3, expr, verbose=verbose)        
printf("AverageVelocity > %d:\n", threshold2)
printRedRatios(dfx3)
printf("Red: Mean correct/predict = %f\n", result2[2])
printf("Blue: Mean correct/predict = %f\n", result2[3])
printf("CV value: %f", result2[1])       
printf("\n\n")